In [ ]:
labels = organize_dir()

In [ ]:
IMG_SIZE = (256,256)
BATCH_SIZE = 32

In [ ]:
def edit_images():
    mask = cv2.imread("../input/masktest/square.jpg", 0)
    mask = cv2.resize(mask, IMG_SIZE)
    
    for dirname, _, filenames in os.walk('./dataset'):
        for filename in filenames:
            fpath = os.path.join(dirname, filename)
            
            img = cv2.imread(fpath)
            img = cv2.resize(img, IMG_SIZE)
            cv2.imwrite(fpath, cv2.bitwise_and(img, img, mask=mask))
            
            del img

In [ ]:
edit_images()

In [ ]:
def load_dataset(img_size, batch_size):
    train = keras.utils.image_dataset_from_directory(
      "./dataset/train",
      seed=123,
      image_size=img_size,
      batch_size=batch_size
    )
    
    test = keras.utils.image_dataset_from_directory(
      "./dataset/test",
      seed=456,
      image_size=img_size,
      batch_size=batch_size
    )
    
    return train, test

In [ ]:
train_ds, test_ds = load_dataset(IMG_SIZE, BATCH_SIZE)

In [ ]:
def show_samples(dataset):
    plt.figure(figsize=(15, 15))
    for images, labels in dataset.take(1):
        for i in range(16):
            ax = plt.subplot(4, 4, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(dataset.class_names[labels[i]])
            plt.axis("off")

In [ ]:
show_samples(train_ds)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def create_model(num_classes):
    model = keras.Sequential([
        keras.layers.Rescaling(1./255, input_shape=(*IMG_SIZE, 3)),
        
        keras.layers.experimental.preprocessing.RandomTranslation(.2, .2),
        keras.layers.experimental.preprocessing.RandomContrast((0, 1)),
        keras.layers.experimental.preprocessing.RandomZoom(.3),
        
        keras.layers.Conv2D(16, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Dropout(.3),
        
        keras.layers.Conv2D(32, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        
        keras.layers.Conv2D(32, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
#         keras.layers.Dropout(.3),
        
        keras.layers.Conv2D(64, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
#         keras.layers.Dropout(.3),
        
        keras.layers.Conv2D(128, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
#         keras.layers.Dropout(.3),
        
        keras.layers.Conv2D(512, (3,3), padding="same", activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
#         keras.layers.Dropout(.1),
        
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation="relu"),
#         keras.layers.Dense(64, activation="relu"),
#         keras.layers.Dropout(.1),
        keras.layers.Dense(num_classes)
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

In [ ]:
NUM_CLASSES = 3
model = create_model(NUM_CLASSES)
model.fit(
    train_ds,
    epochs=10
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
y_pred = model.predict(test_ds)
y_pred_classes = [np.argmax(v) for v in y_pred]
print(classification_report(y_pred_classes, labels["test"]))

In [ ]:
cm = tf.math.confusion_matrix(labels=labels["test"], predictions=y_pred_classes)

plt.figure(figsize=(10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')


In [ ]:
model.layers

In [ ]:
feature_map_model = tf.keras.models.Model(model.input, model.layers[8].output)
feature_map_model.summary()

In [ ]:
img_test = Image.open("./dataset/test/normal/IM-0001-0001.jpeg").convert('RGB')
img_test = img_test.resize(IMG_SIZE)
img_test

In [ ]:
img_test = np.array(img_test)
img_test = img_test[np.newaxis, ...]
img_test.shape

In [ ]:
def plot_features(feature_maps, start, stop):
    for i in range(start, stop):
        plt.figure()
        plt.imshow(feature_maps[0, :, :, i], cmap="gray")

In [ ]:
feature_maps = feature_map_model.predict(img_test)
plot_features(feature_maps, 0, 15)